<a href="https://colab.research.google.com/github/srinithi5011/task_3/blob/main/TASK3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving amazon.csv to amazon.csv


In [ ]:
import sql

In [ ]:
!pip install pandasql
import pandas as pd
from pandasql import sqldf
import io

# Load the uploaded 'amazon.csv' into a pandas DataFrame.
# The `uploaded` variable (from the kernel state) contains the file content.
# We decode it from bytes to string and then read it with pandas.
df_amazon = pd.read_csv(io.StringIO(uploaded['amazon.csv'].decode('utf-8')))

# Assign the DataFrame to a variable named 'PRODUCT' so pandasql can reference it.
PRODUCT = df_amazon

# The SQL query as a string
sql_query = "SELECT * FROM PRODUCT;"

# Execute the SQL query using pandasql. The globals() function makes 'PRODUCT' available.
result = sqldf(sql_query, globals())

# Display the result of the query
print(result)

  Preparing metadata (setup.py) ... done
  Created wheel for pandasql: filename=pandasql-0.7.3-py3-none-any.whl size=26773 sha256=b681f84c5352c124758bdf2878eed440dffe71fdd4ab35c195fd91b32d152c13
  Stored in directory: /root/.cache/pip/wheels/15/a1/e7/6f92f295b5272ae5c02365e6b8fa19cb93f16a537090a1cf27
Successfully built pandasql
      product_id                                       product_name  \
0     B07JW9H4J1  Wayona Nylon Braided USB to Lightning Fast Cha...   
1     B098NS6PVG  Ambrane Unbreakable 60W / 3A Fast Charging 1.5...   
2     B096MSW6CT  Sounce Fast Phone Charging Cable & Data Sync U...   
3     B08HDJ86NZ  boAt Deuce USB 300 2 in 1 Type-C & Micro USB S...   
4     B08CF3B7N1  Portronics Konnect L 1.2M Fast Charging 3A 8 P...   
...          ...                                                ...   
1460  B08L7J3T31  Noir Aqua - 5pcs PP Spun Filter + 1 Spanner | ...   
1461  B01M6453MB  Prestige Delight PRWO Electric Rice Cooker (1 ...   
1462  B009P2LIL4  Bajaj Majesty

In [ ]:
SELECT * FROM PRODUCT ;

SyntaxError: invalid syntax (ipython-input-3797220697.py, line 1)

# Task
Perform SQL operations and aggregate functions, including `SELECT`, `WHERE`, `ORDER BY`, `GROUP BY`, `AVG`, and `COUNT`, on the `PRODUCT` DataFrame. Also, demonstrate `INNER`, `LEFT`, and `RIGHT` joins with the `PRODUCT` DataFrame and a dummy DataFrame. Execute a SQL subquery and simulate SQL views by saving query results into new DataFrames. Additionally, explain how query optimization using indexes can be addressed in `pandasql`.

## Basic SQL Operations & Aggregates

### Subtask:
Demonstrate the use of `SELECT`, `WHERE`, `ORDER BY`, `GROUP BY`, and aggregate functions (like `AVG`, `COUNT`) within a single `pandasql` query. This step will find the average discounted price and product count per category for highly-rated products.


**Reasoning**:
The first instruction is to convert the 'discounted_price' column to a numeric format by removing currency symbols and commas, then casting it to a float, and storing it in a new column named 'discounted_price_numeric'. This prepares the data for aggregation in the SQL query.



In [ ]:
PRODUCT['discounted_price_numeric'] = PRODUCT['discounted_price'].astype(str).str.replace('₹', '').str.replace(',', '').astype(float)
print("Converted 'discounted_price' to numeric 'discounted_price_numeric'. Displaying head of new column:")
print(PRODUCT[['discounted_price', 'discounted_price_numeric']].head())

Converted 'discounted_price' to numeric 'discounted_price_numeric'. Displaying head of new column:
  discounted_price  discounted_price_numeric
0             ₹399                     399.0
1             ₹199                     199.0
2             ₹199                     199.0
3             ₹329                     329.0
4             ₹154                     154.0


**Reasoning**:
The next instruction is to convert the 'rating' column in the PRODUCT DataFrame to a numeric format, coercing any errors to NaN. This ensures that only valid numeric ratings are used for comparison and can be updated in the original column.



In [ ]:
PRODUCT['rating'] = pd.to_numeric(PRODUCT['rating'], errors='coerce')
print("Converted 'rating' column to numeric, coercing errors. Displaying head of updated column:")
print(PRODUCT['rating'].head())

Converted 'rating' column to numeric, coercing errors. Displaying head of updated column:
0    4.2
1    4.0
2    3.9
3    4.2
4    4.2
Name: rating, dtype: float64


**Reasoning**:
The next step is to construct the SQL query string with `SELECT`, `WHERE`, `ORDER BY`, `GROUP BY`, `AVG`, and `COUNT` operations as specified, and then execute it using `pandasql.sqldf`.



In [ ]:
sql_query = """SELECT category, AVG(discounted_price_numeric) AS average_discounted_price, COUNT(product_id) AS product_count FROM PRODUCT WHERE rating >= 4.0 GROUP BY category ORDER BY average_discounted_price DESC;"""
result_df = sqldf(sql_query, globals())
print("Results of the SQL query showing average discounted price and product count per category for highly-rated products:")
print(result_df)

Results of the SQL query showing average discounted price and product count per category for highly-rated products:
                                              category  \
0    Home&Kitchen|Heating,Cooling&AirQuality|AirCon...   
1     Computers&Accessories|Laptops|TraditionalLaptops   
2                        Computers&Accessories|Tablets   
3    Electronics|HomeTheater,TV&Video|Televisions|S...   
4    Home&Kitchen|Kitchen&HomeAppliances|Vacuum,Cle...   
..                                                 ...   
182  Electronics|Mobiles&Accessories|MobileAccessor...   
183  Computers&Accessories|Accessories&Peripherals|...   
184  OfficeProducts|OfficePaperProducts|Paper|Stati...   
185  Home&Kitchen|CraftMaterials|DrawingMaterials|D...   
186  Electronics|Mobiles&Accessories|MobileAccessor...   

     average_discounted_price  product_count  
0                42990.000000              1  
1                37247.000000              1  
2                26999.000000              1  

## Demonstrate SQL JOINS

### Subtask:
Demonstrate INNER, LEFT, and RIGHT joins using the `PRODUCT` DataFrame and a newly created dummy DataFrame. This will illustrate how to combine data from multiple sources.


### Create a dummy DataFrame for Join Operations

To demonstrate SQL JOIN operations, a new DataFrame named `CATEGORY_INFO` will be created. This DataFrame will contain a subset of categories present in the `PRODUCT` DataFrame along with corresponding descriptions. This will allow for clear illustration of `INNER`, `LEFT`, and `RIGHT` join behaviors.

**Reasoning**:
To prepare for the SQL join operations, I will create the `CATEGORY_INFO` dummy DataFrame, populating it with sample category data and descriptions.



In [ ]:
data = {
    'category': [
        'Computers&Accessories|Accessories&Peripherals|Cables&Accessories|Cables|USBCables',
        'Electronics|HomeTheater,TV&Video|Televisions|SmartTelevisions',
        'Home&Kitchen|Kitchen&HomeAppliances|SmallKitchenAppliances|RiceCookers',
        'Home&Kitchen|Heating,Cooling&AirQuality|RoomHeaters',
        'Electronics|Mobiles&Accessories|Smartwatches',
        'Books|Literature&Fiction|IndianWriting|Fiction'
    ],
    'category_description': [
        'Electronics Accessories',
        'Smart Home Entertainment',
        'Kitchen Appliances',
        'Home Climate Control',
        'Wearable Technology',
        'Fiction Books'
    ]
}
CATEGORY_INFO = pd.DataFrame(data)

# Add some categories that might not match perfectly to show LEFT/RIGHT join effects
CATEGORY_INFO = pd.concat([CATEGORY_INFO, pd.DataFrame({
    'category': ['NonExistentCategory1', 'Electronics|Audio&Video|Headphones|Wireless',
                 'Home&Kitchen|Dining&Entertaining|Serveware|DinnerwareSets'],
    'category_description': ['Unmatched Category', 'Audio Accessories', 'Dining Essentials']
})], ignore_index=True)

print("Created CATEGORY_INFO DataFrame. Displaying head:")
print(CATEGORY_INFO.head())

Created CATEGORY_INFO DataFrame. Displaying head:
                                            category      category_description
0  Computers&Accessories|Accessories&Peripherals|...   Electronics Accessories
1  Electronics|HomeTheater,TV&Video|Televisions|S...  Smart Home Entertainment
2  Home&Kitchen|Kitchen&HomeAppliances|SmallKitch...        Kitchen Appliances
3  Home&Kitchen|Heating,Cooling&AirQuality|RoomHe...      Home Climate Control
4       Electronics|Mobiles&Accessories|Smartwatches       Wearable Technology


**Reasoning**:
Now that the `CATEGORY_INFO` DataFrame is created, I will perform an INNER JOIN between `PRODUCT` and `CATEGORY_INFO` on the `category` column as specified in the subtask.



In [ ]:
sql_query_inner_join = """SELECT P.*, C.category_description FROM PRODUCT AS P INNER JOIN CATEGORY_INFO AS C ON P.category = C.category;"""
inner_join_result = sqldf(sql_query_inner_join, globals())
print("Results of INNER JOIN (first 5 rows):")
print(inner_join_result.head())

Results of INNER JOIN (first 5 rows):
   product_id                                       product_name  \
0  B07JW9H4J1  Wayona Nylon Braided USB to Lightning Fast Cha...   
1  B098NS6PVG  Ambrane Unbreakable 60W / 3A Fast Charging 1.5...   
2  B096MSW6CT  Sounce Fast Phone Charging Cable & Data Sync U...   
3  B08HDJ86NZ  boAt Deuce USB 300 2 in 1 Type-C & Micro USB S...   
4  B08CF3B7N1  Portronics Konnect L 1.2M Fast Charging 3A 8 P...   

                                            category discounted_price  \
0  Computers&Accessories|Accessories&Peripherals|...             ₹399   
1  Computers&Accessories|Accessories&Peripherals|...             ₹199   
2  Computers&Accessories|Accessories&Peripherals|...             ₹199   
3  Computers&Accessories|Accessories&Peripherals|...             ₹329   
4  Computers&Accessories|Accessories&Peripherals|...             ₹154   

  actual_price discount_percentage  rating rating_count  \
0       ₹1,099                 64%     4.2       24,269

**Reasoning**:
Now I will perform a LEFT JOIN between `PRODUCT` and `CATEGORY_INFO` on the `category` column to demonstrate how it includes all rows from the left table and matching rows from the right table, or NULLs if no match is found.



In [ ]:
sql_query_left_join = """SELECT P.*, C.category_description FROM PRODUCT AS P LEFT JOIN CATEGORY_INFO AS C ON P.category = C.category;"""
left_join_result = sqldf(sql_query_left_join, globals())
print("Results of LEFT JOIN (first 5 rows):")
print(left_join_result.head())

Results of LEFT JOIN (first 5 rows):
   product_id                                       product_name  \
0  B07JW9H4J1  Wayona Nylon Braided USB to Lightning Fast Cha...   
1  B098NS6PVG  Ambrane Unbreakable 60W / 3A Fast Charging 1.5...   
2  B096MSW6CT  Sounce Fast Phone Charging Cable & Data Sync U...   
3  B08HDJ86NZ  boAt Deuce USB 300 2 in 1 Type-C & Micro USB S...   
4  B08CF3B7N1  Portronics Konnect L 1.2M Fast Charging 3A 8 P...   

                                            category discounted_price  \
0  Computers&Accessories|Accessories&Peripherals|...             ₹399   
1  Computers&Accessories|Accessories&Peripherals|...             ₹199   
2  Computers&Accessories|Accessories&Peripherals|...             ₹199   
3  Computers&Accessories|Accessories&Peripherals|...             ₹329   
4  Computers&Accessories|Accessories&Peripherals|...             ₹154   

  actual_price discount_percentage  rating rating_count  \
0       ₹1,099                 64%     4.2       24,269 

**Reasoning**:
To complete the join operations, I will perform a RIGHT JOIN. Since `pandasql` often implements RIGHT JOIN by swapping tables for a LEFT JOIN, I will perform a LEFT JOIN with `CATEGORY_INFO` as the left table and `PRODUCT` as the right table, joining on the `category` column. This will demonstrate the behavior of a RIGHT JOIN.



In [ ]:
sql_query_right_join = """SELECT P.*, C.category_description FROM CATEGORY_INFO AS C LEFT JOIN PRODUCT AS P ON C.category = P.category;"""
right_join_result = sqldf(sql_query_right_join, globals())
print("Results of RIGHT JOIN (simulated with LEFT JOIN, first 5 rows):")
print(right_join_result.head())

Results of RIGHT JOIN (simulated with LEFT JOIN, first 5 rows):
   product_id                                       product_name  \
0  B008FWZGSG  Samsung Original Type C to C Cable - 3.28 Feet...   
1  B008FWZGSG  Samsung Original Type C to C Cable - 3.28 Feet...   
2  B00GE55L22  Storite USB 3.0 Cable A to Micro B high Speed ...   
3  B00GGGOYEK  Storite USB 2.0 A to Mini 5 pin B Cable for Ex...   
4  B00GGGOYEU  Storite USB 2.0 A to Mini 5 pin B Cable for Ex...   

                                            category discounted_price  \
0  Computers&Accessories|Accessories&Peripherals|...             ₹599   
1  Computers&Accessories|Accessories&Peripherals|...             ₹599   
2  Computers&Accessories|Accessories&Peripherals|...             ₹299   
3  Computers&Accessories|Accessories&Peripherals|...             ₹259   
4  Computers&Accessories|Accessories&Peripherals|...             ₹299   

  actual_price discount_percentage  rating rating_count  \
0         ₹599               

## Execute SQL Subqueries

### Subtask:
Write a `pandasql` query that incorporates a subquery to demonstrate how to use the result of one query as part of another. Specifically, find products with a discounted price higher than the average discounted price of a predefined category.


**Reasoning**:
I need to define a specific category for the subquery, construct a SQL query that uses a subquery to find products with a discounted price higher than the average discounted price of that specific category, and then execute it using `pandasql.sqldf`.



In [ ]:
specific_category = 'Computers&Accessories|Accessories&Peripherals|Cables&Accessories|Cables|USBCables'

sql_subquery = f"""SELECT *
FROM PRODUCT
WHERE discounted_price_numeric > (
    SELECT AVG(discounted_price_numeric)
    FROM PRODUCT
    WHERE category = '{specific_category}'
);
"""

products_above_category_avg = sqldf(sql_subquery, globals())

print(f"Products with discounted price higher than the average of '{specific_category}':")
print(products_above_category_avg.head())

Products with discounted price higher than the average of 'Computers&Accessories|Accessories&Peripherals|Cables&Accessories|Cables|USBCables':
   product_id                                       product_name  \
0  B07JW9H4J1  Wayona Nylon Braided USB to Lightning Fast Cha...   
1  B008IFXQFU  TP-Link USB WiFi Adapter for PC(TL-WN725N), N1...   
2  B0B6F7LX4C  MI 80 cm (32 inches) 5A Series HD Ready Smart ...   
3  B08DPLCM6T  LG 80 cm (32 inches) HD Ready Smart LED TV 32L...   
4  B09C6HXFC1  Duracell USB Lightning Apple Certified (Mfi) B...   

                                            category discounted_price  \
0  Computers&Accessories|Accessories&Peripherals|...             ₹399   
1  Computers&Accessories|NetworkingDevices|Networ...             ₹499   
2  Electronics|HomeTheater,TV&Video|Televisions|S...          ₹13,999   
3  Electronics|HomeTheater,TV&Video|Televisions|S...          ₹13,490   
4  Computers&Accessories|Accessories&Peripherals|...             ₹970   

  actual_

## Simulate SQL Views

### Subtask:
Simulate SQL views by saving the results of two different SQL queries into new DataFrames (`top_rated_products` and `electronics_products`). Then, query these new DataFrames to demonstrate how they can be used like views.

#### Instructions
1. Create a "view" called `top_rated_products` by selecting products from the `PRODUCT` DataFrame that have a `rating` of 4.5 or higher and a `discount_percentage` greater than or equal to '50%'.
2. Create a second "view" called `electronics_products` by selecting products from the `PRODUCT` DataFrame where the `category` starts with 'Electronics'.

**Reasoning**:
Before filtering products by `discount_percentage`, I need to convert the `discount_percentage` column from a string (e.g., '64%') to a numeric format (float).



In [ ]:
PRODUCT['discount_percentage_numeric'] = PRODUCT['discount_percentage'].astype(str).str.replace('%', '').astype(float) / 100
print("Converted 'discount_percentage' to numeric 'discount_percentage_numeric'. Displaying head of new column:")
print(PRODUCT[['discount_percentage', 'discount_percentage_numeric']].head())

Converted 'discount_percentage' to numeric 'discount_percentage_numeric'. Displaying head of new column:
  discount_percentage  discount_percentage_numeric
0                 64%                         0.64
1                 43%                         0.43
2                 90%                         0.90
3                 53%                         0.53
4                 61%                         0.61


**Reasoning**:
Now that the `discount_percentage` is numeric, I will create the `top_rated_products` view by selecting products with a rating of 4.5 or higher and a discount percentage of 50% or more, storing the result in a new DataFrame.



In [ ]:
sql_top_rated = """SELECT * FROM PRODUCT WHERE rating >= 4.5 AND discount_percentage_numeric >= 0.50;"""
top_rated_products = sqldf(sql_top_rated, globals())
print("Created 'top_rated_products' view (DataFrame). Displaying head:")
print(top_rated_products.head())

Created 'top_rated_products' view (DataFrame). Displaying head:
   product_id                                       product_name  \
0  B00NH11PEY  AmazonBasics USB 2.0 - A-Male to A-Female Exte...   
1  B00NH11KIK  AmazonBasics USB 2.0 Cable - A-Male to B-Male ...   
2  B0B86CDHL1  oraimo 65W Type C to C Fast Charging Cable USB...   
3  B0BMXMLSMM  Lapster 65W compatible for OnePlus Dash Warp C...   
4  B09TT6BFDX  Cotbolt Silicone Protective Case Cover for LG ...   

                                            category discounted_price  \
0  Computers&Accessories|Accessories&Peripherals|...             ₹199   
1  Computers&Accessories|Accessories&Peripherals|...             ₹209   
2  Computers&Accessories|Accessories&Peripherals|...             ₹349   
3  Computers&Accessories|Accessories&Peripherals|...             ₹199   
4  Electronics|HomeTheater,TV&Video|Accessories|R...             ₹399   

  actual_price discount_percentage  rating rating_count  \
0         ₹750               

**Reasoning**:
Now I will create the `electronics_products` view by selecting products from the `PRODUCT` DataFrame where the `category` starts with 'Electronics', storing the result in a new DataFrame.



In [ ]:
sql_electronics = """SELECT * FROM PRODUCT WHERE category LIKE 'Electronics%';"""
electronics_products = sqldf(sql_electronics, globals())
print("Created 'electronics_products' view (DataFrame). Displaying head:")
print(electronics_products.head())

Created 'electronics_products' view (DataFrame). Displaying head:
   product_id                                       product_name  \
0  B07KSMBL2H  AmazonBasics Flexible Premium HDMI Cable (Blac...   
1  B0B6F7LX4C  MI 80 cm (32 inches) 5A Series HD Ready Smart ...   
2  B08DPLCM6T  LG 80 cm (32 inches) HD Ready Smart LED TV 32L...   
3  B085194JFL  tizum HDMI to VGA Adapter Cable 1080P for Proj...   
4  B09F6S8BT6  Samsung 80 cm (32 Inches) Wondertainment Serie...   

                                            category discounted_price  \
0  Electronics|HomeTheater,TV&Video|Accessories|C...             ₹219   
1  Electronics|HomeTheater,TV&Video|Televisions|S...          ₹13,999   
2  Electronics|HomeTheater,TV&Video|Televisions|S...          ₹13,490   
3  Electronics|HomeTheater,TV&Video|Accessories|C...             ₹279   
4  Electronics|HomeTheater,TV&Video|Televisions|S...          ₹13,490   

  actual_price discount_percentage  rating rating_count  \
0         ₹700             

**Reasoning**:
Now that the two 'view' DataFrames (`top_rated_products` and `electronics_products`) have been created, I will demonstrate their use by performing a simple SQL query on `top_rated_products`.



In [ ]:
query_top_rated_view = """SELECT product_name, discounted_price_numeric FROM top_rated_products ORDER BY discounted_price_numeric DESC LIMIT 5;"""
result_top_rated_view = sqldf(query_top_rated_view, globals())
print("Querying 'top_rated_products' view (top 5 by discounted price):")
print(result_top_rated_view)

Querying 'top_rated_products' view (top 5 by discounted price):
                                        product_name  discounted_price_numeric
0  WANBO X1 Pro (Upgraded) | Native 1080P Full HD...                   13990.0
1  Aquadpure Copper + Mineral RO+UV+UF 10 to 12 L...                    4999.0
2  Aqua d pure Active Copper 12-L RO+UV Water Fil...                    4999.0
3  Instant Pot Air Fryer, Vortex 2QT, Touch Contr...                    4995.0
4  KINGONE Wireless Charging Pencil (2nd Generati...                    2599.0


**Reasoning**:
Now I will demonstrate how to query the `electronics_products` DataFrame, which simulates an SQL view, by selecting the product name and discounted price for the top 5 most expensive electronic products.



In [ ]:
query_electronics_view = """SELECT product_name, discounted_price_numeric FROM electronics_products ORDER BY discounted_price_numeric DESC LIMIT 5;"""
result_electronics_view = sqldf(query_electronics_view, globals())
print("Querying 'electronics_products' view (top 5 by discounted price):")
print(result_electronics_view)

Querying 'electronics_products' view (top 5 by discounted price):
                                        product_name  discounted_price_numeric
0  Sony Bravia 164 cm (65 inches) 4K Ultra HD Sma...                   77990.0
1  OnePlus 163.8 cm (65 inches) U Series 4K LED S...                   61999.0
2  VU 164 cm (65 inches) The GloLED Series 4K Sma...                   54990.0
3  Samsung 138 cm (55 inches) Crystal 4K Neo Seri...                   47990.0
4  LG 139 cm (55 inches) 4K Ultra HD Smart LED TV...                   47990.0


## Address Query Optimization with Indexes

### Subtask:
Explain that while `pandasql` uses SQLite, which handles some indexing internally, direct control over SQL index creation for explicit query optimization is not available through `pandasql`. For very large datasets, optimizing pandas operations or using a full-fledged database might be more suitable.


### Addressing Query Optimization with Indexes in `pandasql`

While `pandasql` provides a convenient way to query pandas DataFrames using SQL syntax, it's important to understand its limitations regarding query optimization and indexing.

1.  **`pandasql` and SQLite's Internal Indexing**: `pandasql` utilizes the SQLite engine for query execution. SQLite, by default, handles some basic indexing internally, particularly for primary keys and unique constraints. When you perform operations like filtering or joining on columns, SQLite's query planner might leverage these internal structures or create temporary indexes if it deems them beneficial for the query's performance. However, this process is automatic and not explicitly controlled by the user through SQL index creation statements.

2.  **No Direct SQL Index Creation**: Within `pandasql`, there is no direct SQL syntax like `CREATE INDEX` that can be used to explicitly create or manage indexes on DataFrame columns to optimize query performance. The `pandasql` layer translates SQL queries into pandas operations, and the underlying SQLite database operates on an in-memory representation of the DataFrame, which doesn't expose traditional database indexing mechanisms to the user.

3.  **Strategies for Large Datasets**: For very large datasets where query performance becomes critical, relying solely on `pandasql`'s implicit optimizations might not be sufficient. In such scenarios, consider the following:
    *   **Optimize Pandas Operations**: Leverage pandas' native capabilities for performance optimization, such as using vectorized operations, pre-filtering data, or applying efficient data types before passing the DataFrame to `pandasql`. Pandas operations are often highly optimized for data manipulation.
    *   **External Database Systems**: For truly massive datasets or complex analytical workloads, it is generally more suitable to migrate the data to a full-fledged relational database system (e.g., PostgreSQL, MySQL, SQL Server). These databases offer robust indexing capabilities, sophisticated query optimizers, and advanced features for managing and querying large volumes of data, which are not available through `pandasql`.

## Final Task

### Subtask:
Summarize the SQL functionalities demonstrated and provide insights into how to apply them to the dataset, along with a brief explanation of the limitations or alternatives for certain advanced features.


## Summary:

### Q&A
1.  **How can basic SQL operations and aggregate functions be applied to a pandas DataFrame using `pandasql`?**
    Basic SQL operations like `SELECT`, `WHERE`, `ORDER BY`, `GROUP BY`, and aggregate functions (`AVG`, `COUNT`) can be applied by first ensuring data types are numeric (e.g., converting 'discounted\_price' and 'rating' to numeric types). Then, construct a SQL query string and execute it using `sqldf()` against the DataFrame, as demonstrated by finding the average discounted price and product count per category for highly-rated products (rating \$\ge\$ 4.0).

2.  **How are different types of SQL JOINs (INNER, LEFT, RIGHT) performed in `pandasql` with DataFrames?**
    SQL JOINs are performed by constructing a `pandasql` query that specifies the type of join and the columns to join on. For example, an `INNER JOIN` includes only matching rows from both DataFrames, a `LEFT JOIN` includes all rows from the left DataFrame and matching rows from the right (filling non-matches with `NULL`/`NaN`), and a `RIGHT JOIN` can be simulated by reversing the DataFrames in a `LEFT JOIN`. A dummy `CATEGORY_INFO` DataFrame was created to illustrate these joins with the `PRODUCT` DataFrame.

3.  **How can SQL subqueries be executed and SQL views be simulated in `pandasql`?**
    SQL subqueries are executed by embedding one `SELECT` statement within another, such as finding products with a discounted price higher than the average of a specific category. SQL views are simulated by executing a query and saving its result into a new pandas DataFrame. This new DataFrame can then be treated as a "view" and queried further using `pandasql`, as shown with `top_rated_products` and `electronics_products` DataFrames.

4.  **How is query optimization using indexes addressed in `pandasql`?**
    `pandasql` uses SQLite internally, which handles some basic indexing automatically (especially for primary keys). However, direct control over SQL index creation (`CREATE INDEX`) is not available. For query optimization on very large datasets, it is recommended to optimize underlying pandas operations (e.g., vectorized operations, pre-filtering) or to use a full-fledged external database system that provides robust indexing and query optimization features.

### Data Analysis Key Findings
*   **Data Preparation for Aggregation**: Successful conversion of `discounted_price` to `discounted_price_numeric` (float) and `rating` to numeric enabled complex SQL aggregations.
*   **Aggregated Data for Highly-Rated Products**: The query identified categories of highly-rated products, showing varying average discounted prices and product counts. For instance, "Home&Kitchen|Heating,Cooling&AirQuality|AirCon..." had an average discounted price of \$42,990.00 with 1 product meeting the criteria.
*   **SQL Join Demonstrations**:
    *   **INNER JOIN**: Effectively combined `PRODUCT` and `CATEGORY_INFO` DataFrames, retaining only rows where 'category' matched in both.
    *   **LEFT JOIN**: Preserved all rows from the `PRODUCT` DataFrame, appending `category_description` where matches existed and `NaN` otherwise.
    *   **RIGHT JOIN (Simulated)**: Demonstrated how to retain all categories from `CATEGORY_INFO`, appending `PRODUCT` details where available and `NaN` for non-matching products.
*   **Subquery Functionality**: A `pandasql` subquery successfully identified products with a `discounted_price_numeric` exceeding the average discounted price of a `specific_category`.
*   **Simulated SQL Views**: New DataFrames like `top_rated_products` (rating \$\ge\$ 4.5 and discount \$\ge\$ 50%) and `electronics_products` (category starting with 'Electronics') were successfully created and queried like SQL views, showcasing the ability to layer queries.
*   **Indexing Limitations**: `pandasql` does not support explicit SQL index creation; its optimization relies on SQLite's internal handling.

### Insights or Next Steps
*   `pandasql` provides a powerful and intuitive way to leverage SQL syntax for complex data manipulation and aggregation directly on pandas DataFrames, bridging the gap for SQL-savvy users.
*   For datasets where performance is critical or for highly complex querying, consider optimizing operations natively in pandas or migrating data to a dedicated SQL database management system, as `pandasql`'s internal SQLite engine has limitations regarding explicit indexing and advanced query optimization.
